In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import run_eval,train,prepare
from models.baseline_cnn import GCAModel

hparams = {
    'mode':'demo',
    'name':'baseline-cnn',
    'train_embedding':False,
    'batch_size':16,
    'title_size':20,
    'his_size':30,
    'npratio':4,
    'dropout_p':0.2,
    'query_dim':200,
    'embedding_dim':300,
    'filter_num':400,
    'value_dim':16,
    'head_num':16,
    'epochs':5,
    'metrics':'group_auc,ndcg@5,ndcg@10,mean_mrr',
    'device':'cuda:0',
    'attrs': ['title'],
}
save_path = 'models/model_params/{}_{}_{}'.format(hparams['name'],hparams['mode'],hparams['epochs']) +'.model'
device = torch.device(hparams['device']) if torch.cuda.is_available() else torch.device("cpu")

In [2]:
vocab, loader_train, loader_test, loader_validate = prepare(hparams, validate=True)

In [3]:
gcaModel = GCAModel(vocab=vocab,hparams=hparams).to(device).train()

In [4]:
train(gcaModel, hparams, loader_train, loader_test, save_path, loader_validate)

training...
0it [00:02, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 302.00 MiB (GPU 0; 11.75 GiB total capacity; 9.50 GiB already allocated; 70.75 MiB free; 9.59 GiB reserved in total by PyTorch)